In [9]:
import os, json, random
import pandas as pd
from datetime import datetime

import importlib
import ollama

import question_parser, ego_expansion_builder
importlib.reload(question_parser)
importlib.reload(ego_expansion_builder)
from question_parser import QuestionParser
from ego_expansion_builder import EgoExpansionCausalBuilder

from llm_predictors_quiet import (
    predict_meta_informed_llm_reflective,
    predict_combined_context_llm_reflective,
)

MODEL = 'gemma2:27b'
MAX_EXPANSION_DEPTH = 2
MAX_NEIGHBORS_PER_SEED = 5
MAX_RELATIONS_PER_ENTITY = 5

PARSER = QuestionParser(model_name=MODEL, verbose=False)  # 禁用日志输出
BUILDER = EgoExpansionCausalBuilder(
    model_name=MODEL,
    max_neighbors_per_seed=MAX_NEIGHBORS_PER_SEED,
    max_expansion_depth=MAX_EXPANSION_DEPTH,
    max_relations_per_entity=MAX_RELATIONS_PER_ENTITY,
    verbose=False,  # 禁用日志输出
)
print('Config ready: quiet reflective predictors will be used.')

Config ready: quiet reflective predictors will be used.


In [4]:
# Load a local WIQA subset from wiqa_train_data.json (NDJSON).
def load_wiqa_local(path='wiqa_train_data.json', limit=10, seed=42):
    items = []
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line)
            except Exception:
                continue
            q = obj.get('question_stem') or obj.get('question') or ''
            lbl = (obj.get('answer_label') or obj.get('label') or '').strip().lower()
            if q and lbl:
                items.append({'question': q, 'gold': lbl})
    random.Random(seed).shuffle(items)
    return items[:limit]

SAMPLES = load_wiqa_local(limit=1)
len(SAMPLES)


1

In [5]:
def norm(lbl):
    m = {
        'no effect': 'no_effect',
        'no_effect': 'no_effect',
        'more': 'more',
        'less': 'less'
    }
    return m.get((lbl or '').strip().lower())

In [11]:
for i, ex in enumerate(SAMPLES):
    print(f"\n=== Sample {i + 1} ===")
    question = ex['question']
    gold = norm(ex['gold'])
    choices = ex.get('choices')
    print(f"Question: {question}")
    print(f"Gold answer: {gold}")

    r1 = predict_meta_informed_llm_reflective(question, PARSER, BUILDER, MODEL)

 



=== Sample 1 ===
Question: suppose getting a storm over the coast from the ocean happens, how will it affect MORE erosion by the ocean.
Gold answer: more


In [15]:
pred1 = norm(r1.get('final_answer'))

In [16]:
pred1

'more'

In [17]:
r1.get('draft_label')

'more'

In [18]:
r1.get('rationale')

'Storms generate high waves and strong winds, which significantly increase the force of water hitting the coast. This intensified wave action leads to greater erosion of coastal landforms.'